In [ ]:
from __future__ import division
%pylab inline
from scipy import stats
from angles import r2d, r2arcs, d2arcs, arcs2r
def arcm2r(theta):
    return arcs2r(theta*60)
import seaborn as sns;sns.set_style('darkgrid')
import lsst.sims.maf.stackers as stackers
import treecorr as tr
%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams['figure.figsize'] = 19,14
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db
from collections import defaultdict
import treecorr
from itertools import chain
sns.set_context('poster',font_scale=1.4)

In [ ]:
moonperfilter = {}
for colorfilter in ['u', 'g', 'r', 'i', 'z', 'y']:
    outDir='temp'
    conn = db.OpsimDatabase('simruns/minion_1012_sqlite.db')
    bundleList = []
    sql = 'filter = "'+colorfilter+'"'
    metric = metrics.CountMetric('moonAlt')
    slicer = slicers.HealpixSlicer()
    bundleList.append(metricBundles.MetricBundle(metric, slicer, sql))
    bd = metricBundles.makeBundlesDictFromList(bundleList)
    mbg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, saveEarly= False)
    mbg.runAll()
    conn.close()
    moonperfilter[colorfilter] = mbg.simData['moonAlt']

In [ ]:
for colorfilter in moonperfilter.keys():
    m = 0
    for i in moonperfilter[colorfilter]:
        if i<0:
            m+=1
    plt.hist(moonperfilter[colorfilter], label=colorfilter+' {:.3f}'.format(m/len(moonperfilter[colorfilter])),alpha=0.3)#, histtype='step')
plt.legend()

In [ ]:
- query the database for moon_alt, ra, dec
- for each ra, dec, if moonAlt>0: dict[(ra,dec)]+=1
- plot weighted 2d histogram

In [ ]:
outDir='temp'
conn = db.OpsimDatabase('simruns/minion_1012_sqlite.db')
bundleList = []
sql = 'fieldRA > {} and fieldRA < {} and fieldDec > {} and fieldDec < {} '.format(np.radians(55),
                                                                                  np.radians(200),
                                                                                  np.radians(-55),
                                                                                  np.radians(-5))
metric = metrics.PassMetric(('moonAlt', 'filter'))
slicer = slicers.HealpixSlicer(latLonDeg=False)
dith_stack = stackers.NormAirmassStacker(degrees=False)
bundleList.append(metricBundles.MetricBundle(metric, slicer, sql))#, stackerList=[dith_stack]))
bd = metricBundles.makeBundlesDictFromList(bundleList)
mbg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, saveEarly= False)
mbg.runAll()
conn.close()
minion_corrs_per_filter = {}
minion_means = {}
from collections import namedtuple
filters = ['u','g','r','i','z','y']
moon_tuple = namedtuple('moon_tuple', ['visible', 'nonvisible'])
moon_dict = {k:[moon_tuple(0,0)] for k in filters}
filter_data = mbg.simData['filter']
moonAlt = np.array([0 if i<0 else 1 for i in mbg.simData['moonAlt']])
fieldRA = mbg.simData['fieldRA']
fieldDec = mbg.simData['fieldDec']
for filt in filters:
    plt.figure(figsize=(16,12))
    plt.title(filt)
    hist = np.histogram2d(fieldRA[filter_data==filt], fieldDec[filter_data==filt], weights=moonAlt[filter_data==filt],
                           bins=20)
    H, xedges, yedges = hist
    H = H.T
    meanH = np.mean(H)
    H = H - meanH
    minion_means[filt] = meanH
    X = np.array(list(hist[1])*10)
    Y = np.array([[i]*10 for i in hist[2]]).flatten()
    Cat = treecorr.Catalog(k=H.flatten(), ra=X, dec=Y, ra_units='radians', dec_units='radians')
    min_sep = 0.5
    nbins = 20
    max_sep = 50
    Corr = treecorr.KKCorrelation(min_sep=min_sep, max_sep=max_sep, nbins=nbins, sep_units='degrees')
    Corr.process(Cat)
    minion_corrs_per_filter[filt] = Corr
    plt.imshow(H, interpolation='bilinear', origin='low',
         extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
#    plt.hist2d(fieldRA[filter_data==filt], fieldDec[filter_data==filt], weights=moonAlt[filter_data==filt], bins=20);
    plt.colorbar()
#    print(maxlim)
#    plt.clim(-100,100)
    plt.show()

In [ ]:
for filt in filters:
    corr = minion_corrs_per_filter[filt]
    plt.plot(corr.meanr, corr.xi, label=filt)
    # plt.xscale('log')
    plt.legend()



In [ ]:
outDir='temp'
conn = db.OpsimDatabase('simruns/alt_sched.db')
bundleList = []
sql = 'fieldRA > 55 and fieldRA < 200 and fieldDec > -55 and fieldDec < -5 '
metric = metrics.PassMetric(('moonAlt', 'filter'))
slicer = slicers.HealpixSlicer()#latLonDeg=False)
dith_stack = stackers.NormAirmassStacker()#degrees=False)
bundleList.append(metricBundles.MetricBundle(metric, slicer, sql))#, stackerList=[dith_stack]))
bd = metricBundles.makeBundlesDictFromList(bundleList)
mbg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, saveEarly= False)
mbg.runAll()
conn.close()
alt_corrs_per_filter = {}
alt_means = {}
from collections import namedtuple
filters = ['u','g','r','i','z','y']
moon_tuple = namedtuple('moon_tuple', ['visible', 'nonvisible'])
moon_dict = {k:[moon_tuple(0,0)] for k in filters}
filter_data = mbg.simData['filter']
moonAlt = np.array([0 if i<0 else 1 for i in mbg.simData['moonAlt']])
fieldRA = mbg.simData['fieldRA']
fieldDec = mbg.simData['fieldDec']
for filt in ['u','g','r','i','z','y']:
    plt.figure(figsize=(16,12))

    hist = np.histogram2d(fieldRA[filter_data==filt], fieldDec[filter_data==filt], weights=moonAlt[filter_data==filt],
                           bins=20)
    H, xedges, yedges = hist
    H = H.T
    meanH = np.mean(H)
    alt_means[filt] = meanH
    H = H - np.mean(H)
    X = np.array(list(hist[1])*10)
    Y = np.array([[i]*10 for i in hist[2]]).flatten()
    Cat = treecorr.Catalog(k=H.flatten(), ra=X, dec=Y, ra_units='radians', dec_units='radians')
    min_sep = 0.5
    nbins = 20
    max_sep = 50
    Corr = treecorr.KKCorrelation(min_sep=min_sep, max_sep=max_sep, nbins=nbins, sep_units='degrees')
    Corr.process(Cat)
    plt.title('filter: '+filt+', mean: {:.4f}'.format(meanH))
    alt_corrs_per_filter[filt] = Corr
    plt.imshow(H, interpolation='bilinear', origin='low',
         extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
#    plt.hist2d(fieldRA[filter_data==filt], fieldDec[filter_data==filt], weights=moonAlt[filter_data==filt], bins=20);
    plt.colorbar()
#    print(maxlim)
#    plt.clim(-100,100)
    plt.show()

In [ ]:
outDir='temp'
conn = db.OpsimDatabase('simruns/feature_baseline_update_10yrsv2')
bundleList = []
sql = 'proposalId = 0 and fieldRA > 55 and fieldRA < 200 and fieldDec > -55 and fieldDec < -5 '
metric = metrics.PassMetric(('moonAlt', 'filter', 'proposalId'))
slicer = slicers.HealpixSlicer()#latLonDeg=False)
dith_stack = stackers.NormAirmassStacker()#degrees=False)
bundleList.append(metricBundles.MetricBundle(metric, slicer, sql))#, stackerList=[dith_stack]))
bd = metricBundles.makeBundlesDictFromList(bundleList)
mbg = metricBundles.MetricBundleGroup(bd, conn, outDir=outDir, saveEarly= False)
mbg.runAll()
conn.close()
fb_corrs_per_filter = {}
fb_means = {}
from collections import namedtuple
filters = ['u','g','r','i','z','y']
moon_tuple = namedtuple('moon_tuple', ['visible', 'nonvisible'])
moon_dict = {k:[moon_tuple(0,0)] for k in filters}
filter_data = mbg.simData['filter']
moonAlt = np.array([0 if i<0 else 1 for i in mbg.simData['moonAlt']])
fieldRA = mbg.simData['fieldRA']
fieldDec = mbg.simData['fieldDec']
for filt in ['u','g','r','i','z','y']:
    plt.figure(figsize=(16,12))

    hist = np.histogram2d(fieldRA[np.logical_and(filter_data==filt, mbg.simData['proposalId']==0)], fieldDec[np.logical_and(filter_data==filt, mbg.simData['proposalId']==0)], weights=moonAlt[np.logical_and(filter_data==filt, mbg.simData['proposalId']==0)],
                         bins=20)
    H, xedges, yedges = hist
    H = H.T
    meanH = np.mean(H)
    fb_means[filt] = meanH
    H = H - np.mean(H)
    X = np.array(list(hist[1])*10)
    Y = np.array([[i]*10 for i in hist[2]]).flatten()
    Cat = treecorr.Catalog(k=H.flatten(), ra=X, dec=Y, ra_units='radians', dec_units='radians')
    min_sep = 0.5
    nbins = 20
    max_sep = 50
    Corr = treecorr.KKCorrelation(min_sep=min_sep, max_sep=max_sep, nbins=nbins, sep_units='degrees')
    Corr.process(Cat)
    fb_corrs_per_filter[filt] = Corr
    plt.title('filter: '+filt+', mean: {:.4f}'.format(meanH))
    plt.imshow(H, interpolation='bilinear', origin='low',
         extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
#    plt.hist2d(fieldRA[filter_data==filt], fieldDec[filter_data==filt], weights=moonAlt[filter_data==filt], bins=20);
    plt.colorbar()
#    print(maxlim)
#    plt.clim(-100,100)
    plt.show()

In [ ]:
labels = ['minion', 'alt_sched', 'fb']
labels2 = [minion_means, alt_means, fb_means]
for filt in filters:
    plt.figure()
    plt.title(filt)
    for i, corrs in enumerate([minion_corrs_per_filter, alt_corrs_per_filter, fb_corrs_per_filter]):
        corr = corrs[filt]
        plt.plot(corr.meanr, corr.xi, '--', label=labels[i]+' {:.3f} '.format(labels2[i][filt]))
        plt.xscale('log')
        plt.legend()
        plt.yscale('log')